# Code for visualizing match situations, and for calculating and visualizing pitch control and PCV

In [1]:
#parameters used in the Spearman 2018 paper, used for evaluating and calculating the pitch control model
def default_model_params(time_to_control_veto=3):
    #For time to control veto: if the probability that a player can reach the ball and control it is less then 10^-3. Then this player gets ignored
    params = {}
    # model parameters
    params['reaction_time'] = 0.21 # seconds, time to react and change direction. Number taken from Eroglu 2006
    params['tti_sigma'] = 0.45 # Standard deviation of sigmoid function in Spearman 2018 ('s') that determines uncertainty in player arrival time
    params['kappa_def'] =  1. # kappa parameter in Spearman 2018 (=1.72 in the paper) that gives the advantage defending players to control ball. For now, it is set to 1 so that home & away players have same ball control probability
    params['lambda_att'] = 4.3 # ball control parameter for attacking team (Spearman 2018)
    params['lambda_def'] = 4.3 * params['kappa_def'] # ball control parameter for defending team (Spearman 2018)
    params['lambda_gk'] = params['lambda_def']*3.0 #Goalkeepers can much quicker control the ball, since they can also catch the ball in the box
    params['average_ball_speed'] = 22.5 #average ball speed
    params['int_dt'] = 0.04 # integration timestep (dt)
    params['max_int_time'] = 10 # upper limit on player reaching the ball
    params['model_converge_tol'] = 0.01 # assume convergence when PPCF>0.99 at a given location.
    #Short-cut parameters, so that we don't have to calculate pitch control when one player has a sufficient head start. 
    # A sufficient head start is when the a player arrives at the target location at least 'time_to_control' seconds before the next player
    params['time_to_control_att'] = time_to_control_veto*np.log(10) * (np.sqrt(3)*params['tti_sigma']/np.pi + 1/params['lambda_att'])
    params['time_to_control_def'] = time_to_control_veto*np.log(10) * (np.sqrt(3)*params['tti_sigma']/np.pi + 1/params['lambda_def'])
    return params

In [2]:
#find the shirt numbers of the goalkeepers for playing direction and offside purposes
def find_goalkeeper(hometeam, awayteam):
    hometeam = hometeam.iloc[0]
    awayteam = awayteam.iloc[0]
    home_players = []
    away_players = []
    GK_id = []
    #For both teams, find the player with the highest abslute x-value at kick-off. This will be the keeper
    #GK_id will return the shirt numbers of the keepers
    for i in range(1,12):
        hometeam['p'+ str(i) + '_x'] = hometeam['p'+ str(i) + '_x'].astype(np.float64)
        home_players.append(abs(hometeam['p'+ str(i) + '_x']))        
    GK_x = max(home_players)
    for i in range(1,12):
        if abs(hometeam['p'+ str(i) + '_x']) == (GK_x):
            GK_id.append(hometeam['p'+ str(i) + '_shirt_number'])
    for i in range(1,12):
        awayteam['p'+ str(i) + '_x'] = awayteam['p'+ str(i) + '_x'].astype(np.float64)
        away_players.append(abs(awayteam['p'+ str(i) + '_x']))
    GK_x = max(away_players)
    for i in range(1,12):
        if abs(awayteam['p'+ str(i) + '_x']) == (GK_x):
            GK_id.append(awayteam['p'+ str(i) + '_shirt_number'])
    return GK_id

In [3]:
#create a class for each home player for conveniency
class player_home(object):
    
    
    def __init__(self,pid,hometeam, params, GKid):
        self.id = pid #player id
        self.is_gk = self.id == GKid #check if this player is the goalkeeper
        self.playername = "%s_" % (pid) #player id used for functions 
        self.teamname = 'H' #hometeam indicator
        self.reaction_time = params['reaction_time'] #as taken from Eroglu 2006
        self.tti_sigma = params['tti_sigma'] # standard deviation of sigmoid function (see Eq 4 in Spearman, 2018)
        self.lambda_att = params['lambda_att'] # ball control parameter for attacking team (Spearman 2018)
        self.lambda_def = params['lambda_gk'] if self.is_gk else params['lambda_def'] ## ball control parameter for defending team (Spearman 2018). as mentioned earlier, goalkeeper can much quicker control the ball, because he can also catch it in the box
        self.get_position(hometeam) #x and y position of a player
        self.get_velocity(hometeam) #velocity of a player
        self.get_shirt_number(hometeam) #shirt number of the player
        self.get_vmax(hometeam) #maximum speed of the player
        self.PPCF = 0. #for later use
        
    #function to get the shirt number based on the player id   
    def get_shirt_number(self, hometeam):
        self.shirt_number = hometeam[self.playername+'shirt_number']
    
    #function to get the maximum speed based on the shirt number
    def get_vmax(self, hometeam):
        self.vmax = hometeam[self.playername+'max_speed']
        
    #function to get the x and y position of a player
    def get_position(self,hometeam):
        self.position = np.array( [ hometeam[self.playername+'x'], hometeam[self.playername+'y'] ] )
        
    #Function to get the x and y velocity of a player
    def get_velocity(self,hometeam):
        self.velocity = np.array( [ hometeam[self.playername+'vx'], hometeam[self.playername+'vy'] ] )
        if np.any( np.isnan(self.velocity) ):
            self.velocity = np.array([0.,0.])
    
    #function to calculate how much time it would take a player to intercept a pass
    def simple_time_to_intercept(self, r_final):
        self.PPCF = 0. #for later use
        # Time to intercept assumes that the player continues moving at current velocity for 'reaction_time' seconds
        # and then runs at their own maximum speed to the target position.
        r_reaction = self.position + self.velocity*self.reaction_time
        self.time_to_intercept = self.reaction_time + np.linalg.norm((r_final-r_reaction)/100)/self.vmax
        return self.time_to_intercept
    
    #function to calculate the probability that a player can intercept the ball
    def probability_intercept_ball(self,T):
        # probability of a player arriving at target location at time 'T' given their expected time_to_intercept (time of arrival), as described in Spearman 2018
        f = 1/(1. + np.exp( -np.pi/np.sqrt(3.0)/self.tti_sigma * (T-self.time_to_intercept) ) )
        return f

#Create a same class for each away player
class player_away(object):
    
    

    def __init__(self,pid,awayteam, params, GKid):
        self.id = pid
        self.is_gk = self.id == GKid
        self.playername = "%s_" % (pid)
        self.teamname = 'A'
        self.reaction_time = params['reaction_time'] 
        self.tti_sigma = params['tti_sigma'] 
        self.lambda_att = params['lambda_att'] 
        self.lambda_def = params['lambda_gk'] if self.is_gk else params['lambda_def'] 
        self.get_position(awayteam)
        self.get_shirt_number(awayteam)
        self.get_velocity(awayteam)
        self.get_vmax(awayteam)
        self.PPCF = 0. 
    
    def get_shirt_number(self, awayteam):
        self.shirt_number = awayteam[self.playername+'shirt_number']
    
    def get_vmax(self, awayteam):
        self.vmax = awayteam[self.playername+'max_speed']
        
    def get_position(self,awayteam):
        self.position = np.array( [ awayteam[self.playername+'x'], awayteam[self.playername+'y'] ] )
        
    def get_velocity(self,awayteam):
        self.velocity = np.array( [ awayteam[self.playername+'vx'], awayteam[self.playername+'vy'] ] )
        if np.any( np.isnan(self.velocity) ):
            self.velocity = np.array([0.,0.])
    
    def simple_time_to_intercept(self, r_final):
        self.PPCF = 0.
        r_reaction = self.position + self.velocity*self.reaction_time
        self.time_to_intercept = self.reaction_time + np.linalg.norm((r_final-r_reaction)/100)/self.vmax
        return self.time_to_intercept

    def probability_intercept_ball(self,T):
        f = 1/(1. + np.exp( -np.pi/np.sqrt(3.0)/self.tti_sigma * (T-self.time_to_intercept) ) )
        return f


In [4]:
#initialise the classes and create a list of all the home players
def initialise_home_players(hometeam, params,GKid):

    player_ids = []
    for i in range(1,12):
        player_ids.append('p' + str(i))
    home_players = []
    for p in player_ids:
        #
        home_player = player_home(p,hometeam, params,GKid)
        home_players.append(home_player)
    return home_players

#initiate the classes and create a list for all the away players
def initialise_away_players(awayteam, params,GKid):

    player_ids = []
    for i in range(1,12):
        player_ids.append('p' + str(i))
    away_players = []
    for p in player_ids:
        away_player = player_away(p,awayteam, params,GKid)
        away_players.append(away_player)
    return away_players

In [5]:
#function for checking if a player of the attacking team is offside
#this player won't be taken into account when calculating pitch control
def check_offsides( attacking_players, defending_players, ball_start_pos, GK_numbers, tol=0.2):
   
    # find shirt number of defending goalkeeper to find the playing direction
    defending_GK_number = GK_numbers[1] if attacking_players[0].teamname=='H' else GK_numbers[0]
    defending_GK_id = [p.id for p in defending_players if p.shirt_number == defending_GK_number]
    # make sure defending goalkeeper is actually on the field!
    assert defending_GK_id[0] in [p.id for p in defending_players], "Defending goalkeeper shirt number not found in defending players"
    # get goalkeeper player object
    defending_GK = [p for p in defending_players if p.id==defending_GK_id[0]]
    # use defending goalkeeper x-position to figure out which half he is defending (-1: left goal, +1: right goal)
    defending_half = np.sign(defending_GK[0].position[0])
    # find the x-position of the second-deepest defeending player (including GK)
    second_deepest_defender_x = sorted( [defending_half*p.position[0] for p in defending_players], reverse=True )[1]
    # define offside line as being the maximum of second_deepest_defender_x, ball position and half-way line
    #tolerance of 0.2 for inconsistencies in the data
    offside_line = max(second_deepest_defender_x,defending_half*ball_start_pos[0],0.0)+tol
    # any attacking players with x-position greater than the offside line are offside
    attacking_players = [p for p in attacking_players if p.position[0]*defending_half<=offside_line]
    return attacking_players

In [6]:
#import needed packages
from matplotlib import pyplot as plt
from numpy import mean
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.collections import PatchCollection
import matplotlib.patheffects as path_effects
import numpy as np
import xml.etree.ElementTree as et

In [7]:
#draw a pitch for plotting frames and ptich control
def draw_pitch(metadata_pitch, dpi=100, pitch_color='green', zones = False, zones_annotate = False, playing_direction = 'Right'):

    #define the colors of the pitch  
    if pitch_color == 'green':
        pitch_color = '#a8bc95'
        edge_color = 'white'
        face_color = 'white'
        other_color = 'w'
    elif pitch_color == 'white':
        pitch_color = '#ffffff'
        edge_color = 'black'
        face_color = 'black'
        other_color = 'black'
    
    #find the pitch dimensions in the metadata file and initiate the figure
    tree = et.ElementTree(file = metadata_pitch)
    meta_data_pitch = tree.getroot()
    dictionary = [elem.attrib for elem in meta_data_pitch.iter()]
    pitch_x_size = float(dictionary[1].get('fPitchXSizeMeters'))*100
    pitch_y_size = float(dictionary[1].get('fPitchYSizeMeters'))*100
    scalers = np.array([float(pitch_x_size)/100, float(pitch_y_size)/100])
    fig, axes = plt.subplots(figsize=(12.8, 7.2), dpi=dpi)
    fig.patch.set_facecolor(pitch_color)
    
    #define all the pitch lines
    box_height = (16*2 + 7.32)/(float(pitch_y_size)/70)*10000
    box_width = 16/(float(pitch_x_size)/105)*10000
    goal_height = 7.32*100
    goal_area_height = 5.4864*2/(float(pitch_y_size)/110)*10000
    goal_area_width = 5.4864/(float(pitch_x_size)/105)*10000
    D_length = 7/(float(pitch_y_size)/75)*10000
    D_radius = 10/(float(pitch_x_size)/105)*10000
    D_pos = 11.3/(float(pitch_x_size)/80)*10000
    penalty_spot = 11/(float(pitch_x_size)/100)*10000

    
    axes.set_axis_off()
    axes.set_facecolor(pitch_color)
    axes.xaxis.set_visible(False)
    axes.yaxis.set_visible(False)

    #setting the limits of the figure
    axes.set_xlim(-(float(pitch_x_size)/2) - 200, float(pitch_x_size)/2 + 200)
    axes.set_ylim(-(float(pitch_y_size)/2) - 200, float(pitch_y_size)/2 + 200)

    plt.xlim([-(float(pitch_x_size)/2) - 200, float(pitch_x_size)/2 + 200])
    plt.ylim([-(float(pitch_y_size)/2) - 200, float(pitch_y_size)/2 + 200])

    fig.tight_layout(pad=3)

    #draw the pitch boundaries
    axes.add_patch(plt.Rectangle((-(float(pitch_x_size)/2), -(float(pitch_y_size)/2)), pitch_x_size, pitch_y_size, edgecolor=edge_color, facecolor="none"))

    #draw the half-way line
    axes.add_line(plt.Line2D([0, 0], [pitch_y_size/2, -(pitch_y_size/2)], c=edge_color))

    #draw the penalty area
    axes.add_patch(plt.Rectangle(((-pitch_x_size/2), (-box_height/2)),  box_width, box_height, ec=edge_color, fc='none'))
    axes.add_patch(plt.Rectangle(((pitch_x_size/2 - box_width), (-box_height/2)),  box_width, box_height, ec=edge_color, fc='none'))
    #draw the goal area
    axes.add_patch(plt.Rectangle(((pitch_x_size/2)-goal_area_width, (-goal_area_height)/2),  goal_area_width, goal_area_height,
                       ec=edge_color, fc='none'))
    axes.add_patch(plt.Rectangle(((-pitch_x_size/2), (-goal_area_height/2)),  goal_area_width, goal_area_height,
                               ec=edge_color, fc='none'))
    #draw the goals
    axes.add_patch(plt.Rectangle(((pitch_x_size/2), -goal_height/2),  100, goal_height, ec=edge_color, fc='none'))
    axes.add_patch(plt.Rectangle(((-pitch_x_size/2), -goal_height/2),  -100, goal_height, ec=edge_color, fc='none'))


    #draw the halfway circle
    axes.add_patch(Ellipse((0, 0), 2*8/(float(pitch_y_size)/75)*10000, 2*8/(float(pitch_y_size)/75)*10000,
                                    ec=edge_color, fc='none'))
    axes.add_patch(Ellipse((0, 0), 2*8/(float(pitch_y_size)/75)*500, 2*8/(float(pitch_y_size)/75)*500,
                                    ec=edge_color, fc=face_color))
    y = np.linspace(-1,1,50)*D_length # D_length is the chord of the circle that defines the D
    x = np.sqrt(D_radius**2-y**2)+D_pos +100
    axes.plot(pitch_x_size/2-x,y, color = other_color)
    y = np.linspace(-1,1,50)*D_length # D_length is the chord of the circle that defines the D
    x = np.sqrt(D_radius**2-y**2)+D_pos +100
    axes.plot(-pitch_x_size/2+x,y, color = other_color)
    
    #draw the penalty spots
    axes.scatter(pitch_x_size/2-penalty_spot,0.0,marker='o',color = other_color, s = 20)
    axes.scatter(-pitch_x_size/2+penalty_spot,0.0,marker='o',color = other_color, s = 20)
    
    #define the zones for further implementation. (zone 14 is only for attacking purposes)
    zone1 = [[-pitch_x_size/2, -pitch_x_size/2 + box_width], [box_height/2, pitch_y_size/2]]
    zone2 = [[-pitch_x_size/2 + box_width, (-pitch_x_size/2 + box_width)/2], [box_height/2, pitch_y_size/2]]
    zone3 = [[(-pitch_x_size/2 + box_width)/2, 0], [box_height/2, pitch_y_size/2]]
    zone4 = [[0, (pitch_x_size/2 - box_width)/2], [box_height/2, pitch_y_size/2]]
    zone5 = [[(pitch_x_size/2 - box_width)/2, pitch_x_size/2 - box_width], [box_height/2, pitch_y_size/2]]
    zone6 = [[pitch_x_size/2 - box_width, pitch_x_size/2], [box_height/2, pitch_y_size/2]]
    zone7 = [[-pitch_x_size/2, -pitch_x_size/2 + box_width], [goal_area_height/2, box_height/2]]
    zone8 = [[-pitch_x_size/2 + box_width, (-pitch_x_size/2 + box_width)/2], [goal_area_height/2, box_height/2]]
    zone9 = [[(-pitch_x_size/2 + box_width)/2, 0], [goal_area_height/2, box_height/2]]
    zone10 = [[0, (pitch_x_size/2 - box_width)/2], [goal_area_height/2, box_height/2]]
    zone11 = [[(pitch_x_size/2 - box_width)/2, pitch_x_size/2 - box_width], [goal_area_height/2, box_height/2]]
    zone12 = [[pitch_x_size/2 - box_width, pitch_x_size/2], [goal_area_height/2, box_height/2]]
    zone13 = [[-pitch_x_size/2, -pitch_x_size/2 + box_width], [-goal_area_height/2, goal_area_height/2]]
    if playing_direction == 'Right':
        zone14 = [[pitch_x_size/2 - box_width - (((pitch_x_size/2 - box_width) - ((pitch_x_size/2 - box_width)/2))/3), (pitch_x_size/2 - box_width)], [-goal_area_height/2, goal_area_height/2]]
        zone15 = [[-pitch_x_size/2 + box_width, (-pitch_x_size/2 + box_width)/2], [-goal_area_height/2, goal_area_height/2]]        
        zone18 = [[(pitch_x_size/2 - box_width)/2, (pitch_x_size/2 - box_width) - (((pitch_x_size/2 - box_width) - ((pitch_x_size/2 - box_width)/2))/3)], [-goal_area_height/2, goal_area_height/2]]
    else:
        zone14 = [[-pitch_x_size/2 + box_width, (-pitch_x_size/2 + box_width) - (((-pitch_x_size/2 + box_width) - ((-pitch_x_size/2 + box_width)/2))/3)], [-goal_area_height/2, goal_area_height/2]]
        zone15 = [[(-pitch_x_size/2 + box_width) - (((-pitch_x_size/2 + box_width) - ((-pitch_x_size/2 + box_width)/2))/3), (-pitch_x_size/2 + box_width)/2], [-goal_area_height/2, goal_area_height/2]]
        zone18 = [[(pitch_x_size/2 - box_width)/2, pitch_x_size/2 - box_width], [-goal_area_height/2, goal_area_height/2]]
    zone16 = [[(-pitch_x_size/2 + box_width)/2, 0], [-goal_area_height/2, goal_area_height/2]]
    zone17 = [[0, (pitch_x_size/2 - box_width)/2], [-goal_area_height/2, goal_area_height/2]]
    zone19 = [[pitch_x_size/2 - box_width, pitch_x_size/2], [-goal_area_height/2, goal_area_height/2]]
    zone20 = [[-pitch_x_size/2, -pitch_x_size/2 + box_width], [-box_height/2, -goal_area_height/2]]
    zone21 = [[-pitch_x_size/2 + box_width, (-pitch_x_size/2 + box_width)/2], [-box_height/2, -goal_area_height/2]]
    zone22 = [[(-pitch_x_size/2 + box_width)/2, 0], [-box_height/2, -goal_area_height/2]]
    zone23 = [[0, (pitch_x_size/2 - box_width)/2], [-box_height/2, -goal_area_height/2]]
    zone24 = [[(pitch_x_size/2 - box_width)/2, pitch_x_size/2 - box_width], [-box_height/2, -goal_area_height/2]]
    zone25 = [[pitch_x_size/2 - box_width, pitch_x_size/2], [-box_height/2, -goal_area_height/2]]
    zone26 = [[-pitch_x_size/2, -pitch_x_size/2 + box_width], [-pitch_y_size/2, -box_height/2]]
    zone27 = [[-pitch_x_size/2 + box_width, (-pitch_x_size/2 + box_width)/2], [-pitch_y_size/2, -box_height/2]]
    zone28 = [[(-pitch_x_size/2 + box_width)/2, 0], [-pitch_y_size/2, -box_height/2]]
    zone29 = [[0, (pitch_x_size/2 - box_width)/2], [-pitch_y_size/2, -box_height/2]]
    zone30 = [[(pitch_x_size/2 - box_width)/2, pitch_x_size/2 - box_width], [-pitch_y_size/2, -box_height/2]]
    zone31 = [[pitch_x_size/2 - box_width, pitch_x_size/2], [-pitch_y_size/2, -box_height/2]]
    zones_complete = [zone1, zone2, zone3, zone4, zone5, zone6, zone7, zone8, zone9, zone10, zone11, zone12, zone13, zone14, zone15, zone16, zone17, zone18, zone19, zone20, zone21, zone22, zone23, zone24, zone25, zone26, zone27, zone28, zone29, zone30, zone31]
    
    #draw the zones
    if zones:
        axes.add_line(plt.Line2D([-pitch_x_size/2 + box_width, -pitch_x_size/2 + box_width], [pitch_y_size/2, -(pitch_y_size/2)], c=edge_color, linestyle="dashed"))
        axes.add_line(plt.Line2D([pitch_x_size/2 - box_width, pitch_x_size/2 - box_width], [pitch_y_size/2, -(pitch_y_size/2)], c=edge_color, linestyle="dashed"))
        if playing_direction == 'Right':
            axes.add_line(plt.Line2D([(pitch_x_size/2 - box_width) - (((pitch_x_size/2 - box_width) - ((pitch_x_size/2 - box_width)/2))/3), (pitch_x_size/2 - box_width) - (((pitch_x_size/2 - box_width) - ((pitch_x_size/2 - box_width)/2))/3)], [-goal_area_height/2, goal_area_height/2], c=edge_color, linestyle="dashed"))
        else:
            axes.add_line(plt.Line2D([(-pitch_x_size/2 + box_width) - (((-pitch_x_size/2 + box_width) - ((-pitch_x_size/2 + box_width)/2))/3), (-pitch_x_size/2 + box_width) - (((-pitch_x_size/2 + box_width) - ((-pitch_x_size/2 + box_width)/2))/3)], [-goal_area_height/2, goal_area_height/2], c=edge_color, linestyle="dashed"))
        axes.add_line(plt.Line2D([(-pitch_x_size/2 + box_width)/2, (-pitch_x_size/2 + box_width)/2], [pitch_y_size/2, -(pitch_y_size/2)], c=edge_color, linestyle="dashed"))
        axes.add_line(plt.Line2D([(pitch_x_size/2 - box_width)/2, (pitch_x_size/2 - box_width)/2], [pitch_y_size/2, -(pitch_y_size/2)], c=edge_color, linestyle="dashed"))  
        axes.add_line(plt.Line2D([-pitch_x_size/2, pitch_x_size/2], [-box_height/2 , -box_height/2], c=edge_color, linestyle="dashed"))   
        axes.add_line(plt.Line2D([-pitch_x_size/2, pitch_x_size/2], [box_height/2 , box_height/2], c=edge_color, linestyle="dashed")) 
        axes.add_line(plt.Line2D([-pitch_x_size/2, pitch_x_size/2], [-goal_area_height/2 , -goal_area_height/2], c=edge_color, linestyle="dashed"))   
        axes.add_line(plt.Line2D([-pitch_x_size/2, pitch_x_size/2], [goal_area_height/2 , goal_area_height/2], c=edge_color, linestyle="dashed")) 
    
    #add zone numbers to figure
    if zones_annotate:
        axes.text(mean(zone1[0]), mean(zone1[1]), "1", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone2[0]), mean(zone2[1]), "2", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone3[0]), mean(zone3[1]), "3", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone4[0]), mean(zone4[1]), "4", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone5[0]), mean(zone5[1]), "5", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone6[0]), mean(zone6[1]), "6", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone7[0]), mean(zone7[1]), "7", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone8[0]), mean(zone8[1]), "8", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone9[0]), mean(zone9[1]), "9", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone10[0]), mean(zone10[1]), "10", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone11[0]), mean(zone11[1]), "11", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone12[0]), mean(zone12[1]), "12", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone13[0]), mean(zone13[1]), "13", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone14[0]), mean(zone14[1]), "14", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone15[0]), mean(zone15[1]), "15", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone16[0]), mean(zone16[1]), "16", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone17[0]), mean(zone17[1]), "17", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone18[0]), mean(zone18[1]), "18", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone19[0]), mean(zone19[1]), "19", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone20[0]), mean(zone20[1]), "20", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone21[0]), mean(zone21[1]), "21", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone22[0]), mean(zone22[1]), "22", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone23[0]), mean(zone23[1]), "23", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone24[0]), mean(zone24[1]), "24", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone25[0]), mean(zone25[1]), "25", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone26[0]), mean(zone26[1]), "26", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone27[0]), mean(zone27[1]), "27", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone28[0]), mean(zone28[1]), "28", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone29[0]), mean(zone29[1]), "29", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone30[0]), mean(zone30[1]), "30", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
        axes.text(mean(zone31[0]), mean(zone31[1]), "31", horizontalalignment='center', verticalalignment='center', fontsize=14, color = face_color)
    return fig, axes, zones_complete

In [8]:
#same as previous function. However, this one looks worse but can be used for creating videos of the frames
#importing important packages
from matplotlib import pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.collections import PatchCollection
import matplotlib.patheffects as path_effects
import numpy as np
import xml.etree.ElementTree as et

def plot_pitch( metadata_pitch, field_color ='green', linewidth=2, markersize=20):
    
    #find the pitch boundaries in the metadata file
    tree = et.ElementTree(file = metadata_pitch)
    meta_data_pitch = tree.getroot()
    dictionary = [elem.attrib for elem in meta_data_pitch.iter()]
    pitch_x_size = float(dictionary[1].get('fPitchXSizeMeters'))*100
    pitch_y_size = float(dictionary[1].get('fPitchYSizeMeters'))*100
    
    fig,ax = plt.subplots(figsize=(12,8)) # create a figure 
    #define field colors
    if field_color=='green':
        ax.set_facecolor('mediumseagreen')
        lc = 'whitesmoke' # line color
        pc = 'w' # 'spot' colors
    elif field_color=='white':
        lc = 'k'
        pc = 'k'
    
    #define pitch lines
    border_dimen = (3,3) # include a border around the field of width 3m
    half_pitch_length = float(pitch_x_size)/2. # length of half pitch
    half_pitch_width = float(pitch_y_size)/2. # width of half pitch
    signs = [-1,1] 
    # Soccer field dimensions typically defined in yards, so we need to convert to meters
    box_height = (16*2 + 7.32)/(float(pitch_y_size)/70)*10000
    box_width = 16/(float(pitch_x_size)/105)*10000
    goal_height = 7.32*100
    goal_area_height = 5.4864*2/(float(pitch_y_size)/110)*10000
    goal_area_width = 5.4864/(float(pitch_x_size)/105)*10000
    D_length = 7/(float(pitch_y_size)/75)*10000
    D_radius = 10/(float(pitch_x_size)/105)*10000
    D_pos = 11.3/(float(pitch_x_size)/80)*10000
    penalty_spot = 11/(float(pitch_x_size)/100)*10000
    penalty_spot = 11/(float(pitch_x_size)/100)*10000
    corner_radius = 1/(float(pitch_y_size)/75)*10000
    centre_circle_radius = 2*4/(float(pitch_y_size)/75)*10000
    
    #draw half-way line and circle
    ax.plot([0,0],[-half_pitch_width,half_pitch_width],lc,linewidth=linewidth)
    ax.scatter(0.0,0.0,marker='o',facecolor=lc,linewidth=0,s=markersize)
    y = np.linspace(-1,1,50)*centre_circle_radius
    x = np.sqrt(centre_circle_radius**2-y**2)
    ax.plot(x,y,lc,linewidth=linewidth)
    ax.plot(-x,y,lc,linewidth=linewidth)
    for s in signs: # plots each line seperately
        # draw pitch boundary
        ax.plot([-half_pitch_length,half_pitch_length],[s*half_pitch_width,s*half_pitch_width],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length],[-half_pitch_width,half_pitch_width],lc,linewidth=linewidth)
        # draw posts & line
        ax.plot([s*(half_pitch_length + 200),s*(half_pitch_length + 200)],[-goal_height/2.,goal_height/2.],lc,linewidth=linewidth)
        ax.plot([s*(half_pitch_length + 200),s*(half_pitch_length)],[-goal_height/2.,-goal_height/2.],lc,linewidth=linewidth)
        ax.plot([s*(half_pitch_length + 200),s*(half_pitch_length)],[goal_height/2.,goal_height/2.],lc,linewidth=linewidth)
        #draw goal area
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*goal_area_width],[goal_area_height/2.,goal_area_height/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*goal_area_width],[-goal_area_height/2.,-goal_area_height/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length-s*goal_area_width,s*half_pitch_length-s*goal_area_width],[-goal_area_height/2.,goal_area_height/2.],lc,linewidth=linewidth)
        #draw penalty area
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*box_width],[box_height/2.,box_height/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*box_width],[-box_height/2.,-box_height/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length-s*box_width,s*half_pitch_length-s*box_width],[-box_height/2.,box_height/2.],lc,linewidth=linewidth)
        #draw penalty spot
        ax.scatter(s*half_pitch_length-s*penalty_spot,0.0,marker='o',facecolor=lc,linewidth=0,s=markersize)
        #draw corner flags
        y = np.linspace(0,1,50)*corner_radius
        x = np.sqrt(corner_radius**2-y**2)
        ax.plot(s*half_pitch_length-s*x,-half_pitch_width+y,lc,linewidth=linewidth)
        ax.plot(s*half_pitch_length-s*x,half_pitch_width-y,lc,linewidth=linewidth)
        #draw the D
        y = np.linspace(-1,1,50)*D_length # D_length is the chord of the circle that defines the D
        x = np.sqrt(D_radius**2-y**2)+D_pos +100
        ax.plot(s*half_pitch_length-s*x,y,lc,linewidth=linewidth)
        
    # remove axis labels and ticks
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    # set axis limits
    xmax = pitch_x_size/2 + 700
    ymax = pitch_y_size/2 + 700
    ax.set_xlim([-xmax,xmax])
    ax.set_ylim([-ymax,ymax])
    ax.set_axisbelow(True)
    return fig,ax


In [9]:
#import packages
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
import numpy as np
import xml.etree.ElementTree as et

#plot a single frame
def plot_frame(home_df, away_df, frame_number, metadata_pitch, naming_data, team_colors=('r','b'), pitch_color = 'green', PlayerMarkerSize=12, PlayerAlpha=0.7, include_player_velocities=False, annotate=False, zones = False, zones_annotate = False):
    #extract the row in the home and away dataframes that corresponds to the frame number
    hometeam = home_df.loc[frame_number]
    awayteam = away_df.loc[frame_number]
    tree = et.ElementTree(file = metadata_pitch)
    meta_data_pitch = tree.getroot()
    dictionary = [elem.attrib for elem in meta_data_pitch.iter()]
    pitch_x_size = float(dictionary[1].get('fPitchXSizeMeters'))*100
    pitch_y_size = float(dictionary[1].get('fPitchYSizeMeters'))*100
    #intialise parameters
    params = default_model_params()
    #find goalkeeper shirt numbers
    GK_id = find_goalkeeper(home_df, away_df)
    #initialise attacking and defending players
    if hometeam['Team_in_possession'] == 'H':
        attacking_players = initialise_home_players(hometeam, params, GK_id[0])
        defending_players = initialise_away_players(awayteam, params, GK_id[1])
        
    elif hometeam['Team_in_possession'] == 'A':
        defending_players = initialise_home_players(hometeam, params, GK_id[0])
        attacking_players = initialise_away_players(awayteam, params, GK_id[1])
    defending_GK_number = GK_id[1] if attacking_players[0].teamname=='H' else GK_id[0]
    defending_GK_position = [p.position for p in defending_players if p.shirt_number == defending_GK_number]
    if defending_GK_position[0][0] < 0:
        playing_direction = 'Left'
    elif defending_GK_position[0][0] > 0:
        playing_direction = 'Right'
    #draw the pitch
    fig,ax,zones = draw_pitch(metadata_pitch, pitch_color = pitch_color, zones = zones, zones_annotate = zones_annotate, playing_direction = playing_direction)
    data = et.ElementTree(file = naming_data)
    name_data = data.getroot()
    #add game and time information to the figure
    ax.text(0, pitch_y_size/2 + 400, "{} (red) vs {} (blue)".format(name_data[0].attrib["home_team_name"], name_data[0].attrib["away_team_name"]), fontsize=14, horizontalalignment='center')
    frame_minute =  int( hometeam['Time_in_game']/60. )
    frame_second =  ( awayteam['Time_in_game']/60. - frame_minute ) * 60.
    if hometeam['Half'] == 1:
        timestring = "%d:%1.2f" % ( frame_minute, frame_second  )
        ax.text(0, int(pitch_y_size)/2 + 100, timestring, fontsize=14, horizontalalignment='center' )
    else:
        timestring = "%d:%1.2f" % ( 45 + frame_minute, frame_second  )
        ax.text(0, int(pitch_y_size)/2 + 100, timestring, fontsize=14, horizontalalignment='center' )        
    
    #plot the player positions
    for team,color in zip( [hometeam,awayteam], team_colors) :
        x_columns = [c for c in team.keys() if c[-2:].lower()=='_x'] 
        y_columns = [c for c in team.keys() if c[-2:].lower()=='_y'] 
        ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
        #plot player velocities
        if include_player_velocities:
            vx_columns = ['{}_vx'.format(c[:-2]) for c in x_columns] 
            vy_columns = ['{}_vy'.format(c[:-2]) for c in y_columns] 
            x_values = team[x_columns]
            x_values = pd.to_numeric(x_values, errors='coerce').astype('float64')
            y_values = team[y_columns]
            y_values = pd.to_numeric(y_values, errors='coerce').astype('float64')
            vx_values = team[vx_columns]
            vy_values = team[vy_columns]
            vx_values = pd.to_numeric(vx_values, errors='coerce').astype('float64')
            vy_values = pd.to_numeric(vy_values, errors='coerce').astype('float64')
            ax.quiver( x_values, y_values, vx_values, vy_values, color=color, scale_units='inches', scale=10.,width=0.0015,headlength=5,headwidth=3,alpha=PlayerAlpha)
        #add player shirt numbers to the plot
        if annotate:
            for i in range(1,12):
                hometeam['p'+ str(i) + '_x'] = hometeam['p'+ str(i) + '_x'].astype(np.float64)
                awayteam['p'+ str(i) + '_x'] = awayteam['p'+ str(i) + '_x'].astype(np.float64)
                hometeam['p'+ str(i) + '_y'] = hometeam['p'+ str(i) + '_y'].astype(np.float64)
                awayteam['p'+ str(i) + '_y'] = awayteam['p'+ str(i) + '_y'].astype(np.float64)
                if hometeam['p'+ str(i) + '_shirt_number'] >= 10:
                    ax.text( hometeam['p'+ str(i) + '_x']-60, hometeam['p'+ str(i) + '_y']-40, hometeam['p'+ str(i) + '_shirt_number'], fontsize=8, color='black'  )
                else:
                    ax.text(hometeam['p'+ str(i) + '_x']-32, hometeam['p'+ str(i) + '_y']-40, hometeam['p'+ str(i) + '_shirt_number'], fontsize=8, color='black'  )
                if awayteam['p'+ str(i) + '_shirt_number'] >= 10:
                    ax.text(awayteam['p'+ str(i) + '_x']-60, awayteam['p'+ str(i) + '_y']-40, awayteam['p'+ str(i) + '_shirt_number'], fontsize=8, color='black'  )
                else: 
                    ax.text(awayteam['p'+ str(i) + '_x']-32, awayteam['p'+ str(i) + '_y']-40, awayteam['p'+ str(i) + '_shirt_number'], fontsize=8, color='black' )
         
    #plot ball
    ax.plot( hometeam['Ball_x_position'], hometeam['Ball_y_position'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
    return fig, ax, zones, playing_direction

In [10]:
#ignore warnings for plotting
import warnings
warnings.filterwarnings("ignore")

In [11]:
#create video of frames. Used the plot pitch function here since the draw pitch figure can't be saved
import matplotlib.animation as manimation
import matplotlib
import cv2
from progressbar import ProgressBar
pbar = ProgressBar()
def save_match_clip(hometeam,awayteam, fpath, meta_data_pitch, naming_data, frame_numbers, fname='clip_test', figax=None, frames_per_second=25, team_colors=('r','b'), include_player_velocities=False, PlayerMarkerSize=12, PlayerAlpha=0.7, pitch_control = False, annotate = False, pitch_color='Green'):


    # Set figure and movie settings
    FFMpegWriter = manimation.writers['ffmpeg']
    metadata = dict(title='Tracking Data', artist='Matplotlib', comment='Penalty moment')
    writer = FFMpegWriter(fps=frames_per_second, metadata=metadata)
    fname = fpath + '/' +  fname + '.mp4' # path and filename
    tree = et.ElementTree(file = meta_data_pitch)
    metadata_pitch_ = tree.getroot()
    dictionary = [elem.attrib for elem in metadata_pitch_.iter()]
    pitch_x_size = float(dictionary[1].get('fPitchXSizeMeters'))*100
    pitch_y_size = float(dictionary[1].get('fPitchYSizeMeters'))*100
    
    # draw the pitch
    if figax is None:
        fig,ax = plot_pitch(metadata_pitch = meta_data_pitch, field_color = pitch_color)
    else:
        fig,ax = figax
    fig.set_tight_layout(True)
    data = et.ElementTree(file = naming_data)
    name_data = data.getroot()
    ax.text(0, pitch_y_size/2 + 400, "{} (red) vs {} (blue)".format(name_data[0].attrib["home_team_name"], name_data[0].attrib["away_team_name"]), fontsize=14, horizontalalignment='center')
    # Generate movie
    print("Generating movie...",end='')
    with writer.saving(fig, fname, 100):
        for i in pbar(frame_numbers):
            figobjs = [] # this is used to collect up all the axis objects so that they can be deleted after each iteration
            #plot player positions
            for team,color in zip( [hometeam.loc[i],awayteam.loc[i]], team_colors) :
                x_columns = [c for c in team.keys() if c[-2:].lower()=='_x'] 
                y_columns = [c for c in team.keys() if c[-2:].lower()=='_y'] 
                objs, = ax.plot( team[x_columns], team[y_columns], color+'o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
                figobjs.append(objs)
                #plot player velocities
                if include_player_velocities:
                    vx_columns = ['{}_vx'.format(c[:-2]) for c in x_columns]
                    vy_columns = ['{}_vy'.format(c[:-2]) for c in y_columns]
                    x_values = team[x_columns]
                    x_values = pd.to_numeric(x_values, errors='coerce').astype('float64')
                    y_values = team[y_columns]
                    y_values = pd.to_numeric(y_values, errors='coerce').astype('float64')
                    vx_values = team[vx_columns]
                    vy_values = team[vy_columns]
                    vx_values = pd.to_numeric(vx_values, errors='coerce').astype('float64')
                    vy_values = pd.to_numeric(vy_values, errors='coerce').astype('float64')
                    objs = ax.quiver( x_values, y_values, vx_values, vy_values, color=color, scale_units='inches', scale=10.,width=0.0015,headlength=5,headwidth=3,alpha=PlayerAlpha)
                    figobjs.append(objs)
            # plot ball
            objs, = ax.plot( hometeam.loc[i]['Ball_x_position'], hometeam.loc[i]['Ball_y_position'], 'ko', MarkerSize=6, alpha=1.0, LineWidth=0)
            figobjs.append(objs)
            
            #add player shirt numbers
            if annotate:
                for j in range(1,12):
                    hometeam.loc[i]['p'+ str(j) + '_x'] = hometeam.loc[i]['p'+ str(j) + '_x'].astype(np.float64)
                    awayteam.loc[i]['p'+ str(j) + '_x'] = awayteam.loc[i]['p'+ str(j) + '_x'].astype(np.float64)
                    hometeam.loc[i]['p'+ str(j) + '_y'] = hometeam.loc[i]['p'+ str(j) + '_y'].astype(np.float64)
                    awayteam.loc[i]['p'+ str(j) + '_y'] = awayteam.loc[i]['p'+ str(j) + '_y'].astype(np.float64)
                    if hometeam.loc[i]['p'+ str(j) + '_shirt_number'] >= 10:
                        objs = ax.text( hometeam.loc[i]['p'+ str(j) + '_x']-60, hometeam.loc[i]['p'+ str(j) + '_y']-40, hometeam.loc[i]['p'+ str(j) + '_shirt_number'], fontsize=8, color='black'  )
                        figobjs.append(objs)
                    else:
                        objs = ax.text(hometeam.loc[i]['p'+ str(j) + '_x']-32, hometeam.loc[i]['p'+ str(j) + '_y']-40, hometeam.loc[i]['p'+ str(j) + '_shirt_number'], fontsize=8, color='black'  )
                        figobjs.append(objs)
                    if awayteam.loc[i]['p'+ str(j) + '_shirt_number'] >= 10:
                        objs = ax.text(awayteam.loc[i]['p'+ str(j) + '_x']-60, awayteam.loc[i]['p'+ str(j) + '_y']-40, awayteam.loc[i]['p'+ str(j) + '_shirt_number'], fontsize=8, color='black'  )
                        figobjs.append(objs)
                    else: 
                        objs = ax.text(awayteam.loc[i]['p'+ str(j) + '_x']-32, awayteam.loc[i]['p'+ str(j) + '_y']-40, awayteam.loc[i]['p'+ str(j) + '_shirt_number'], fontsize=8, color='black' )
                        figobjs.append(objs)
            #for later use when pitch control functions are defined.
            #adds pitch control to the video
            if pitch_control:
                team_in_possession = hometeam.loc[i]['Team_in_possession']
                PPCF_home, PPCF_away, xgrid, ygrid, weighted_PPCF_home, weighted_PPCF_away, weighted_pitch_control_ball_pos_home, weighted_pitch_control_ball_pos_away, transition_layer = generate_pitch_control_for_frame(i, hometeam, awayteam,  params = default_model_params(), GK_id = find_goalkeeper(hometeam, awayteam),  meta_data_pitch = meta_data_pitch, n_grid_cells_x = 50, offsides = True)
                cmap = 'bwr'
                objs = ax.imshow(np.flipud(PPCF_home), extent=(-pitch_x_size/2., pitch_x_size/2., -pitch_y_size/2., pitch_y_size/2.),interpolation='spline36',vmin=0.0,vmax=1.0,cmap=cmap,alpha=0.5)
                figobjs.append(objs)
            # include match time at the top
            frame_minute =  int( team['Time_in_game']/60. )
            frame_second =  ( team['Time_in_game']/60. - frame_minute ) * 60.
            if hometeam.loc[i]['Half'] == 1:
                timestring = "%d:%1.2f" % ( frame_minute, frame_second  )
                objs = ax.text(0,int(pitch_y_size)/2 + 100, timestring, fontsize=14, horizontalalignment='center' )
            else:
                timestring = "%d:%1.2f" % ( frame_minute + 45, frame_second  )
                objs = ax.text(0,int(pitch_y_size)/2 + 100, timestring, fontsize=14, horizontalalignment='center' )
            figobjs.append(objs)
            writer.grab_frame()
            # Delete all axis objects (other than pitch lines) in preperation for next frame
            for figobj in figobjs:
                figobj.remove()
    print("done")
    plt.clf()
    plt.close(fig)

In [12]:
#functions for calculating pitch control
def generate_pitch_control_for_frame(frame_number, homedf, awaydf, params, GK_id, meta_data_pitch, n_grid_cells_x = 50, offsides = False):
    import numpy as np
    import pickle
    import torch
    # get the home and away rows for the frames
    hometeam = homedf.loc[frame_number]
    awayteam = awaydf.loc[frame_number]
    #find out which team is in possession
    team_in_possession = hometeam.loc['Team_in_possession']
    #extract the ball position
    ball_start_pos = np.array([hometeam.loc['Ball_x_position'], hometeam.loc['Ball_y_position']])
    # break the pitch down into a grid
    tree = et.ElementTree(file = meta_data_pitch)
    metadata_pitch_ = tree.getroot()
    dictionary = [elem.attrib for elem in metadata_pitch_.iter()]
    pitch_x_size = float(dictionary[1].get('fPitchXSizeMeters'))*100
    pitch_y_size = float(dictionary[1].get('fPitchYSizeMeters'))*100
    n_grid_cells_y = int(n_grid_cells_x*pitch_y_size/pitch_x_size)
    dx = pitch_x_size/n_grid_cells_x
    dy = pitch_y_size/n_grid_cells_y
    xgrid = np.arange(n_grid_cells_x)*dx - pitch_x_size/2. + dx/2.
    ygrid = np.arange(n_grid_cells_y)*dy - pitch_y_size/2. + dy/2.
    # initialise pitch control grids for attacking and defending teams based on the x and y grid 
    PPCFa = np.zeros( shape = (len(ygrid), len(xgrid)) )
    PPCFd = np.zeros( shape = (len(ygrid), len(xgrid)) )
    # initialise player classes for pitch control calculations
    if team_in_possession=='H':
        attacking_players = initialise_home_players(hometeam, params, GK_id[0])
        defending_players = initialise_away_players(awayteam, params, GK_id[1])
        
    elif team_in_possession=='A':
        defending_players = initialise_home_players(hometeam, params, GK_id[0])
        attacking_players = initialise_away_players(awayteam, params, GK_id[1])
    else:
        assert False, "Team in possession must be either home or away" 
    
    #Remove attacking players that are offside
    if offsides:
        attacking_players = check_offsides( attacking_players, defending_players, ball_start_pos, GK_id)
    # calculate pitch pitch control model at each location on the pitch
    for i in range( len(ygrid) ):
        for j in range( len(xgrid) ):
            target_position = np.array( [xgrid[j], ygrid[i]] )
            PPCFa[i,j],PPCFd[i,j] = calculate_pitch_control_at_target(target_position, attacking_players, defending_players, ball_start_pos, params = default_model_params())
    # check probabilitiy sums within convergence
    checksum = np.sum( PPCFa + PPCFd ) / float(n_grid_cells_y*n_grid_cells_x ) 
    assert 1-checksum < params['model_converge_tol'], "Checksum failed: %1.3f" % (1-checksum)
    
    #assign attacking or defending pitch control to home and away teams
    if team_in_possession=='H':
        PPCF_home = PPCFa
        PPCF_away = PPCFd
    else:
        PPCF_home = PPCFd
        PPCF_away = PPCFa

    device = 'cpu'; dtype = torch.float32
    xy_query = []
    w_PPCF_home = []
    w_PPCF_away = []
    
    #code for weighted pitch control by implementing pitch value. Code partly derive from Will Thompson (twitter)
    #add pitch control to list
    for i in range(len(ygrid)):
        for j in range(len(xgrid)):
            xy_query.append([xgrid[j], ygrid[i]])
            w_PPCF_home.append(PPCF_home[i][j].item())
            w_PPCF_away.append(PPCF_away[i][j].item())
    w_PPCF_home = torch.Tensor(w_PPCF_home)
    w_PPCF_away = torch.Tensor(w_PPCF_away)
    xy_query = torch.Tensor(xy_query)
    #get the position of the ball
    locs_ball = np.asarray(df_home.iloc[:,range(68,70)])
    xy_ball = torch.Tensor([locs_ball[df_home.index.get_loc(frame_number)]])
    #calculate the transition layer that indicates a normal distribution of where the ball could go next
    transition_layer = torch.exp(( - torch.sqrt(torch.sum(((xy_query/100) - (xy_ball/100)) **2,axis = 1))/ (2*14)))
    transition_layer = transition_layer / transition_layer.sum()
    #multiply pitch control with the transition layer and reshape it back to the original shape
    weighted_pitch_control_ball_pos_home = w_PPCF_home * transition_layer
    weighted_pitch_control_ball_pos_home = torch.max(weighted_pitch_control_ball_pos_home,torch.zeros_like(weighted_pitch_control_ball_pos_home))
    weighted_pitch_control_ball_pos_home = np.reshape(weighted_pitch_control_ball_pos_home, (n_grid_cells_y, n_grid_cells_x))
    weighted_pitch_control_ball_pos_away = w_PPCF_home * transition_layer
    weighted_pitch_control_ball_pos_away = torch.max(weighted_pitch_control_ball_pos_away,torch.zeros_like(weighted_pitch_control_ball_pos_away))
    weighted_pitch_control_ball_pos_away = np.reshape(weighted_pitch_control_ball_pos_away, (n_grid_cells_y, n_grid_cells_x))
    transition_layer = np.reshape(transition_layer, (n_grid_cells_y, n_grid_cells_x))        
    #import an expected threat grid
    df_xT = pd.read_csv('xT.csv', header=None)
    xTMap = torch.tensor(df_xT.values)
    xTMap_interp = torch.nn.functional.interpolate(xTMap[None,None,:,:],(n_grid_cells_y,n_grid_cells_x))
    xTflipped = torch.flip(xTMap_interp[0,0],[0,1])
    expected_threat_left_to_right = xTMap_interp[0,0].reshape((1,-1))
    expected_threat_right_to_left = xTflipped.reshape((1,-1))
    #find the playing direction and direct the expected threat grid based on the playing direction
    params = default_model_params()
    GK_id = find_goalkeeper(homedf, awaydf)
    if hometeam['Team_in_possession'] == 'H':
        attacking_players = initialise_home_players(hometeam, params, GK_id[0])
        defending_players = initialise_away_players(awayteam, params, GK_id[1])
        
    elif hometeam['Team_in_possession'] == 'A':
        defending_players = initialise_home_players(hometeam, params, GK_id[0])
        attacking_players = initialise_away_players(awayteam, params, GK_id[1])
    defending_GK_number = GK_id[1] if attacking_players[0].teamname=='H' else GK_id[0]
    defending_GK_position = [p.position for p in defending_players if p.shirt_number == defending_GK_number]
    if defending_GK_position[0][0] < 0:
        playing_direction = 'Left'
    elif defending_GK_position[0][0] > 0:
        playing_direction = 'Right'
    if playing_direction == 'Right':
        xTweights = expected_threat_left_to_right
    else:
        xTweights = expected_threat_right_to_left
    #reshape the weighted pitch control again
    weighted_PPCF_home_new = weighted_pitch_control_ball_pos_home.reshape((1,-1))
    weighted_PPCF_away_new = weighted_pitch_control_ball_pos_away.reshape((1,-1))
    #multiply it by the expected threat grid
    weighted_PPCF_home = xTweights*weighted_PPCF_home_new
    weighted_PPCF_away = xTweights*weighted_PPCF_away_new
    #reshape to the original shapes
    weighted_PPCF_home = np.reshape(weighted_PPCF_home, (n_grid_cells_y, n_grid_cells_x))
    weighted_PPCF_away = np.reshape(weighted_PPCF_away, (n_grid_cells_y, n_grid_cells_x))

    return PPCF_home, PPCF_away, xgrid, ygrid, weighted_PPCF_home, weighted_PPCF_away, weighted_pitch_control_ball_pos_home, weighted_pitch_control_ball_pos_away, transition_layer

#function for calculating the pitch control at the target location on the grid
def calculate_pitch_control_at_target(target_position, attacking_players, defending_players, ball_start_pos, params):
    # calculate ball travel time from the start position to the target position.
    if ball_start_pos is None or any(np.isnan(ball_start_pos)) == True: # assume that ball is already at location
        ball_travel_time = 0.0 
    else:
        # ball travel time is distance to target position from current ball position divided by the average ball speed
        ball_travel_time = np.linalg.norm( (target_position - ball_start_pos)/100 )/params['average_ball_speed']
    
    # first get arrival time of 'nearest' attacking player (nearest also dependent on current velocity)
    tau_min_att = np.nanmin( [p.simple_time_to_intercept(target_position) for p in attacking_players] )
    tau_min_def = np.nanmin( [p.simple_time_to_intercept(target_position ) for p in defending_players] )
    # check whether we actually need to solve equation 3
    if tau_min_att-max(ball_travel_time,tau_min_def) >= params['time_to_control_def']:
        # if defending team can arrive significantly before attacking team, no need to solve pitch control model
        return 0., 1.
    elif tau_min_def-max(ball_travel_time,tau_min_att) >= params['time_to_control_att']:
        # if attacking team can arrive significantly before defending team, no need to solve pitch control model
        return 1., 0.
    else: 
        # solve pitch control model by integrating equation 3 in Spearman et al.
        # first remove any player that is far (in time) from the target location
        attacking_players = [p for p in attacking_players if p.time_to_intercept-tau_min_att < params['time_to_control_att'] ]
        defending_players = [p for p in defending_players if p.time_to_intercept-tau_min_def < params['time_to_control_def'] ]
        # set up integration arrays
        dT_array = np.arange(ball_travel_time-params['int_dt'],ball_travel_time+params['max_int_time'],params['int_dt']) 
        PPCFatt = np.zeros_like( dT_array )
        PPCFdef = np.zeros_like( dT_array )
        # integration equation 3 of Spearman 2018 until convergence or tolerance limit hit (see 'params')
        ptot = 0.0
        i = 1
        while 1-ptot>params['model_converge_tol'] and i<dT_array.size: 
            T = dT_array[i]
            for player in attacking_players:
                # calculate ball control probablity for a player in time interval T+dt
                dPPCFdT = (1-PPCFatt[i-1]-PPCFdef[i-1])*player.probability_intercept_ball( T ) * player.lambda_att
                # make sure it's greater than zero
                assert dPPCFdT>=0, 'Invalid attacking player probability (calculate_pitch_control_at_target)'
                player.PPCF += dPPCFdT*params['int_dt'] # total pitch control contribution of a single player
                PPCFatt[i] += player.PPCF #calculate the sum for the attacking team
            for player in defending_players:
                # calculate ball control probablity for a player in time interval T+dt
                dPPCFdT = (1-PPCFatt[i-1]-PPCFdef[i-1])*player.probability_intercept_ball( T ) * player.lambda_def
                # make sure it's greater than zero
                assert dPPCFdT>=0, 'Invalid defending player probability (calculate_pitch_control_at_target)'
                player.PPCF += dPPCFdT*params['int_dt'] # total pitch conytol contribution of a single player
                PPCFdef[i] += player.PPCF #calculate the sum for the defending team
            ptot = PPCFdef[i]+PPCFatt[i] # total pitch control probability 
            i += 1
        if i>=dT_array.size:
            print("Integration failed to converge: %1.3f" % (ptot) )
        #return attacking and defending pitch control grids 
        return PPCFatt[i-1], PPCFdef[i-1]

In [13]:
from scipy.spatial import distance
#function to find the closest x and y positions in the grid
def closest_nodes(zones, xgrid, ygrid):
    #for the zones, find the closest x and y positions in the grid
    closest_index_x1 = []
    closest_index_x2 = []
    closest_index_y1 = []
    closest_index_y2 = []
    for i in range(len(zones)):
        distances_x1 = []
        for j in range(len(xgrid)):
            distances_x1.append(abs(zones[i][0][0] - xgrid[j]))
        for j in range(len(distances_x1)):
            if distances_x1[j] == min(distances_x1):
                closest_index_x1.append(j)
                break
    for i in range(len(zones)):
        distances_x2 = []
        for j in range(len(xgrid)):
            distances_x2.append(abs(zones[i][0][1] - xgrid[j]))
        for j in range(len(distances_x2)):
            if distances_x2[j] == min(distances_x2):
                closest_index_x2.append(j)
                break
    for i in range(len(zones)):
        distances_y1 = []
        for j in range(len(ygrid)):
            distances_y1.append(abs(zones[i][1][0] - ygrid[j]))
        for j in range(len(distances_y1)):
            if distances_y1[j] == min(distances_y1):
                closest_index_y1.append(j)
                break
    for i in range(len(zones)):
        distances_y2 = []
        for j in range(len(ygrid)):
            distances_y2.append(abs(zones[i][1][1] - ygrid[j]))
        for j in range(len(distances_y2)):
            if distances_y2[j] == min(distances_y2):
                closest_index_y2.append(j)
                break
    return [[closest_index_x1, closest_index_x2], [closest_index_y1, closest_index_y2]]


In [14]:
#function to plot the pitch control for a frame
def plot_pitchcontrol_for_frame(frame_number, homedf, awaydf, naming_data, metadata_pitch, alpha = 0.7, n_grid_cells_x = 50, include_player_velocities=True, annotate=False, zones = False, show_pitch_control = True, zones_annotate = False, annotate_values = False):
    #find pitch boundaries in the metadata file
    tree = et.ElementTree(file = metadata_pitch)
    metadata_pitch_ = tree.getroot()
    dictionary = [elem.attrib for elem in metadata_pitch_.iter()]
    pitch_x_size = float(dictionary[1].get('fPitchXSizeMeters'))*100
    pitch_y_size = float(dictionary[1].get('fPitchYSizeMeters'))*100
    #extract the rows in the home and away dataframe
    hometeam = homedf.loc[frame_number]
    awayteam = awaydf.loc[frame_number]
    team_in_possession = hometeam.loc['Team_in_possession']
    # plot frame and calculate pitch control
    fig,ax, zones, playing_direction = plot_frame(homedf, awaydf, frame_number, naming_data = naming_data, metadata_pitch = metadata_pitch, pitch_color='white', include_player_velocities=include_player_velocities, annotate=annotate, zones = zones, zones_annotate = zones_annotate)
    PPCF_home, PPCF_away, xgrid,ygrid, weighted_PPCF_home, weighted_PPCF_away, weigthed_PPCF_ball_home, weighted_PPCF_ball_away, transition_layer = generate_pitch_control_for_frame(frame_number, homedf, awaydf,  params = default_model_params(), GK_id = find_goalkeeper(homedf, awaydf),  meta_data_pitch = metadata_pitch, n_grid_cells_x = n_grid_cells_x, offsides = True)
    #plot pitch control surface
    if show_pitch_control:
        cmap = 'bwr'
        ax.imshow(np.flipud(PPCF_home), extent=(-pitch_x_size/2., pitch_x_size/2., -pitch_y_size/2., pitch_y_size/2.),interpolation='spline36',vmin=0.0,vmax=1.0,cmap=cmap,alpha=0.6)
    #caluclate pitch control values for the zones and plot them
    if annotate_values:
        closest_nodes_pitch = closest_nodes(zones, xgrid, ygrid)
        zone_values = []
        for i in range(len(zones)):
            values = []
            for j in range(closest_nodes_pitch[1][0][i], closest_nodes_pitch[1][1][i]):
                for x in range(closest_nodes_pitch[0][0][i], closest_nodes_pitch[0][1][i]):
                    values.append(PPCF_home[j][x])
                    ax.text(0, -pitch_y_size/2 - 200,'Pitch cotrol hometeam: {}'.format(round(mean(PPCF_home),3)), horizontalalignment='center', verticalalignment='center', fontsize=12, color = 'black')    
            zone_values.append(round(mean(values), 3))
        for i in range(len(zones)):
            ax.text(mean(zones[i][0]), mean(zones[i][1]), zone_values[i], horizontalalignment='center', verticalalignment='center', fontsize=8, color = 'black')
        return fig, ax, zone_values, PPCF_home, PPCF_away, weighted_PPCF_home, weighted_PPCF_away
    else:
        ax.text(0, -pitch_y_size/2 - 200,'Pitch cotrol hometeam: {}'.format(round(mean(PPCF_home),3)), horizontalalignment='center', verticalalignment='center', fontsize=12, color = 'black')
        return fig, ax, PPCF_home, PPCF_away, weighted_PPCF_home, weighted_PPCF_away

